# RNN (Recurrent Neural Network)

- http://blog.naver.com/PostView.nhn?blogId=magnking&logNo=221323257045&redirect=Dlog&widgetTypeCall=true&directAccess=false
- https://wikidocs.net/47193


[simple RNN]
-  cnn(특징추출) - ffnn
- rnn도 마찬가지로 ffnn해줘야함_ 모든 신경망의 기본은 ffnn

- 입력 데이터 받고 => 가중치 => output : 이 통째로 cell이라한다

- 여러개의 cell을 묶어서 output과 연결시킨다 
- 이전데이터 가중치의 영향을 고려한다  > 앞단의 데이터가 모두 끝나야한다는 것 . 
    - 그래서 latent time(지연시간)  발생 
    - 각 cell의 output이 연속족으로 최종 output까지 영향을 미쳐야하는데
    - 중간에 영향력을 잃거나 중복적으로 영향을 미치는 경우가 있다
    
[Long Short Time Memory]: 장기기억 _ https://wikidocs.net/22888
- 그래서 나온게 LSTM : 처음의 영향력을 잃지않기위해 사용
    - STATE를 2개 준다 : CONTROL STATE , HIDDEN STATE
    - HIDDEN STATE 가 OUTPUT: 지속적으로 다음요소에 영향을 미칠것을 결정?
    - 하나는 다음 회로로 나가고 하나는 output으로 나감?
    - 
- gate 3개 : forget input output
    - forget : 앞단에서 흘러오는 ct-1 를 어떻게 제어할것인가를 결정( 망각 or keep)
        - 현재 들어오는 데이터와 이전 state에서 들어오는 ht-1 을 고려해서 
    - input : control선에 어떻게 변화해야 영향력을 유지할 수 있을지 제어 
    - output : hidden
       
- 출력을 위해서 가중치 4개가 필요 (forget input output 과 CELL 안에서 영향을 끼치는한개(TANH)를 다 계산)
    - 4개의 메모리공간 필요
    - 계속 영향을 미치기는 하지만 가중치가 다름    
    - 가중치 사이즈 계산할 떄 4배해줌
    
    
[ GRU ] : LSTM이 너무 복잡해서 나온것

- FORGET과 INPUT 을 하나로 묶음 :회로를 간략화 , 연산이 줄얻을음
- 순서가 있는 데이터를 다를 수 있다 .-> 시계열 DATA : SEQUENTIAL DATA
    - 나는 /학교에/ 간다 
   
 [CAPTIONING]   첫번째 셀(iNput)에 이미지를 넣어주면 이미지를 captioning하는 것
 
- ONE TO ONE :
- ONE TO MANY : captioning 할 때 사용 
- MANY TO MANY :번역할 때 사용 
- MANY TO MANY :동영상 FRAME에 들어있는 OBJECT를 DETECTION 할 때

[rnn 종류]_ 휴대폰 사진찍어놈
- 홋 RNN: Simple RNN 
- Bi directional RNN: 양쪽방향으로 뽑음
- MULTI RNN:  CELL을 여러개 쌓아서 다층으로_ 다층일수록 정확


#input character => input layer => hidden layer => ouput layer => target character


-CELL
MULTI LSTM CELL

- STATIC RNN :셀 수가 고정
- DYNAMIC RNN : 시간적으로 계산하면되어서 (쭉 늘어놀 필요가 없다 ) 하나계산하고 하나계산하고 쮺쮺



## 예제1

In [2]:
import numpy as np
import os
import tensorflow as tf

tf.reset_default_graph() # 그래프 초기화 (변수생성)

n_inputs =3 #입력데이터
n_neurons =5  #셀의 가중치 사이즈 = 특성을 찾아내는 개수


X0=tf.placeholder(tf.float32,[None,n_inputs]) #4(none)x3(n_inputs)
X1=tf.placeholder(tf.float32,[None,n_inputs])
#[0,1,2] 이거 하나가 한 셀에 입력된다

#FFNN(feed forward neural network)

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units = n_neurons) # num_units ;가중치 사이즈 = 특성을 찾아내는 개수
#static_rnn : rnn network : 4개의 셀이 연결되면서 메모리 확보 
output_seqs,states = tf.contrib.rnn.static_rnn(basic_cell,[X0,X1],dtype=tf.float32) # 2x4x3 #2 :batch size x 4: 셀수 x 3: 뉴런수


Y0,Y1=output_seqs

#출력, 다음 셀로 전달되어지는 값 : 마지막 state값 
init = tf.global_variables_initializer()
# 012 의 특징추출 -> 345 특징추출-> . . . ->901 특징추출 _다음셀로 전달

X0_batch= np.array([[0,1,2],[3,4,5],[6,7,8],[9,0,1]])

X1_batch =np.array([[9,8,7],[0,0,0],[6,5,4],[3,2,1]])

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [3]:
with tf.Session() as sess:
    init.run() #feed _ dict :두개의 데이터를 줌 
    Y0_val,Y1_val = sess.run([Y0,Y1],feed_dict={X0: X0_batch,X1:X1_batch})
print('처음 데이터 특성:',Y0_val,'차수:',Y0_val.shape) 
print('두번째 데이터 특성:',Y1_val,'차수:',Y1_val.shape)

#Y0은 4X5 : CELL이 4개들어갔고  특징이 3개에서 5개로 늘었다  
# 앞에서 설정해준 특성값때문에

# 셀이 가지고 있는 가중치 사이즈는?  3X5  3개로 나와서 5로나갔기때문 ????????

처음 데이터 특성: [[-0.5604998   0.630265   -0.37392837  0.77174777 -0.8781064 ]
 [-0.9075139   0.9996531  -0.24984421  0.9995794  -0.9999721 ]
 [-0.9834446   0.99999976 -0.1169731   0.9999994  -1.        ]
 [ 0.9952488   0.99988854  0.962079    0.9963758  -0.99999714]] 차수: (4, 5)
두번째 데이터 특성: [[-0.9854498   1.          0.63506484  0.99999994 -1.        ]
 [-0.5352604  -0.8014373  -0.083668    0.08273393 -0.1960368 ]
 [-0.9419458   0.99995667  0.49007377  0.9999601  -0.9999989 ]
 [-0.74517167  0.99486434 -0.18341762  0.9828464  -0.94516325]] 차수: (4, 5)


# MNIST
- https://excelsior-cjh.tistory.com/184

In [ ]:
tf.reset_default_graph() #위에거랑 충돌되서 다시 초기화 해줌 

#원래이미지 784(28x28)

n_steps=28 #셀이 28개 
n_inputs=28 # 셀당 `
learning_rate = 0.001

# 3차원으로 바꾼이유 : 
X= tf.placeholder(tf.float32, [None, n_steps, n_inputs]) #None :batchsize
y = tf.placeholder(tf.int32, [None])           

#셀 하나가 ffnn
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons) #150개
# 셀 : 28개 만들어짐

# dense : 
# states: 마지막 셀의 수평으로 전달되는  값   #150개는 특징 추출한거
# states ; 마지막 셀 output과 동일
#28개의 셀이 있는데 마지막 1개의 ouput만 사용 :many to one (감정분류 -> 분류기)
#state의 차수 : 150*150
#output의 차수 : latent time(지연시간)을통해 계산되어진 셀의 모든값을 결합 출력 150x28x150 
# 150x28x150: 150개의 특징중에 10개만 추려내야함
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

#입력차수, 출력차수만 지정하면 자동으로 바이어스를 생성, 가중치 공간 확보 
#state : 150x150 * output:150x10 
#dense 의 가중치 사이즈 = 150x10
logits= tf.layers.dense(states, n_outputs) 

# 분류를 위한 미분이 가능한식 도출 ↓
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss= tf.reduce_mean(xentropy) #배치사이즈 사용 => 지역해의 위험때문에 배치사이즈의 평균사용
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate) #adam은 momentum + propgrad
training_op = optimizer.minimize(loss)
correct= tf.nn.in_top_k(logits,y, 1) # 가장큰값을 취해서  비교 
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
init = tf.global_variables_initializer()

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data 
mnist = input_data.read_data_sets('/tmp/data/')
X_test = mnist.test.images.reshape((-1,n_steps,n_inputs)) # reshape 한 이유는 원래 이미지 모양으로 학습하기위해서 
y_test = mnist.test.labels

In [ ]:
n_epochs = 100
batch_size = 150
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range( #60000(traindata)/150
            mnist.train.num_examples // batch_size):
            X_batch,y_batch = mnist.train.next_batch(batch_size)
            # 이미지 사이즈로 생성
            X_batch = X_batch.reshape((-1,n_steps,n_inputs)) 
            sess.run(training_op,feed_dict ={X:X_batch,y:y_batch})
        
    acc_train=accuracy.eval(feed_dict={X:X_batch,y:y_batch})
    acc_test = accuracy.eval(feed_dict={X:X_test,y:y_test})
    print(epoch,'train accuracy:',acc_train,'test accuracy:',acc_test)
    
    
    #train 99 #test 98

## Multi RNNCell : 

- basic rnn cell
- multi rnn cell :수직으로 layer 구성 : 수직으로 cell
- dynamic rnn cell : model 

In [ ]:
tf.reset_default_graph()
n_steps = 28 #셀이 28개
n_inputs = 28
n_outputs = 10
learning_rate = 0.001
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])
n_neurons = 100 #특징 
n_layers = 3 #3개의 멀티레이어 : 3개의 cell을 생성
layers = [tf.contrib.rnn.BasicRNNCell(num_units = n_neurons,
                                     activation = tf.nn.relu)
         for layer in range(n_layers)]



#3개를 조합해서 multi rnn cell 만듬 
multi_layer_cell =tf.contrib.rnn.MultiRNNCell(layers)


 concat 
     #https://lsjsj92.tistory.com/456

In [ ]:
#dynamic rnn :셀로 입력되는 데이터의 개수에 맞추어서 셀을 구성

#eg 나는 학교에 간다 : 2글자 3글자 2글자 : 세개의 사이즈를 일치시키기 위해 
#static rnn : 큰것 기준으로 맞츠고 작은것은 padding

# 3층 셀이 28개가 조성 : 고정 사이즈
#입력사이즈 변동하고 가중치 조절을 통해서  나가는 특징(neural수)을 일치시킴

outputs,states = tf.nn.dynamic_rnn(multi_layer_cell,X,dtype = tf.float32)

# state 몇개가 발생하는가? : 3(n_layers) x 150(batch size) x 100 (n_neurons)
# 특징을 세번 추출 
states_concat =tf.concat(axis=1,values=states) # axis =1 이기 때문에 가로로 쭉
#가중치 학습 FFNN 망 ↓여기부터↓↓↓
logits = tf.layers.dense(states_concat,n_outputs) #가중치 : 입력과 출력이 주어지면 자기가 알아서 계산함/ 

#가중치 :300x10 
#logits 확률값: 150(미니배치 사이즈)x10(특징 수)
#원핫인코딩 * log(예측값)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)

loss =tf.reduce_mean(xentropy)
optimizer =tf.train.AdamOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits,y,1)
accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))
init = tf.global_variables_initializer()

In [ ]:
n_epochs = 10
batch_size = 150
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(
            mnist.train.num_examples // batch_size):
            X_batch,y_batch = mnist.train.next_batch(batch_size)
            # 이미지 사이즈로 생성
            X_batch = X_batch.reshape((-1,n_steps,n_inputs)) 
            sess.run(training_op,feed_dict ={X:X_batch,y:y_batch})
        
    acc_train=accuracy.eval(feed_dict={X:X_batch,y:y_batch})
    acc_test = accuracy.eval(feed_dict={X:X_test,y:y_test})
    print(epoch,'train accuracy:',acc_train,'test accuracy:',acc_test)
    


## 다른방법으로 
- http://blog.naver.com/PostView.nhn?blogId=magnking&logNo=221323257045&redirect=Dlog&widgetTypeCall=true&directAccess=false

In [ ]:
import tensorflow as tf
from tensorflow.python.framework import ops
ops.reset_default_graph()
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets('./mnist/data/', one_hot=True)

learning_rate = 0.001
total_epoch = 10 #학습횟수
batch_size=128 # 128개의 데이터를 학습하고 난 후 한 번의 가중치 갱신
n_input=28 #한 단계에 입력되는 데이터의 개수

n_step=28
n_hidden=128 #
n_class=10 #분류하고자 하는 분류 카테고리 수이자 출력층의 노드수

# X 사이즈 128x280x28  # Y 사이즈 128x10
X=tf.placeholder(tf.float32, [None,n_step, n_input]) 
Y= tf.placeholder(tf.float32, [None,n_class])
W= tf.Variable(tf.random_normal([n_hidden, n_class])) # W 128x10
b= tf.Variable(tf.random_normal([n_class]))
cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden) #28x128 사이즈의 가중치 자동지정 #neurons : 128특징
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32) #cell 수 : 28 개 

#FFNN 
#한장은 28x128 이고, 이게 128장이 있다 
#output size : 128(batchsize) x 28( cell 수) x 128 ( cell마댜 특징 128)
outputs = tf.transpose(outputs, [1,0,2]) #인덱스로 사이즈 바꾸기 => 28x128x128
outputs = outputs[-1]  # 맨마지막 셀: 128x128 (128셀에대해서 128?)
model = tf.matmul(outputs, W) + b #128x128 *128x10 = 128개의 특징에서 10개로
cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [ ]:
sess= tf.Session()
sess.run(tf.global_variables_initializer())
#60000/128
total_batch = int(mnist.train.num_examples/batch_size)
for epoch in range(total_epoch):
    total_cost = 0
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        #이미지 사이즈로 reshape
        batch_xs = batch_xs.reshape((batch_size, n_step, n_input))
        #경사하강법 
        _, cost_val = sess.run([optimizer, cost], feed_dict={X:batch_xs, Y:batch_ys})#_,: 파이썬이 계산한 마지막 결과가 저장되는것
        total_cost += cost_val
    print("Epoch:", "%04d" % (epoch +1),
          'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))
print("최적화 완료!")

## 문제
- 1. 테스트 데이터를 이용하여 테스트하는 회로를 구성하시오
- 2. 3개의 셀을 갖는 Multi-layer cell로 수정하시오 

In [ ]:
import tensorflow as tf
from tensorflow.python.framework import ops
ops.reset_default_graph()
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets('./mnist/data/', one_hot=True)

learning_rate = 0.001
total_epoch = 10
batch_size=128
n_input=28
n_step=28
n_hidden=128
n_class=10
n_layers=3  # 3개의 멀티레이어 

X=tf.placeholder(tf.float32, [None,n_step, n_input]) # 128x28x28
Y= tf.placeholder(tf.float32, [None,n_class]) # 128x10
W= tf.Variable(tf.random_normal([n_hidden, n_class])) # 128x10
b= tf.Variable(tf.random_normal([n_class])) # 10

# 3개의 셀로 멀티레이어 생성해도 output은 하나의 셀일 경유와 동일
# state에서는 3개의 레이어마다 생성
layers=[tf.contrib.rnn.BasicRNNCell(num_units=n_hidden, activation=tf.nn.relu) for layer in range(n_layers)]  

multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers) 


outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)

outputs = tf.transpose(outputs, [1,0,2]) 

outputs = outputs[-1] 

model = tf.matmul(outputs, W) + b # 128x128, 128x10, 10 => 128x10
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [ ]:
sess= tf.Session() # 위에서 with 없이 열었음


In [ ]:
is_correct = tf.equal(tf.argmax(model,1),tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct,tf.float32))
test_batch_size = len(mnist.test.images)
test_xs = mnist.test.images.reshape(test_batch_size, n_step,n_input)

test_ys = mnist.test.labels
print('정확도:',sess.run(accuracy,feed_dict={X:test_xs,Y:test_ys}))

In [ ]:
# 규진언니것
import tensorflow as tf
from tensorflow.python.framework import ops
ops.reset_default_graph()
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)
learning_rate = 0.001
total_epoch = 10
batch_size = 128
n_input = 28
n_step = 28
n_hidden = 128
n_class = 10
X = tf.placeholder(tf.float32, [None, n_step, n_input]) # 128X28X28
Y = tf.placeholder(tf.float32, [None, n_class]) # 128X10
W = tf.Variable(tf.random_normal([n_hidden, n_class])) #128X10
b = tf.Variable(tf.random_normal([n_class])) # 10

n_layers = 3
layers = [tf.contrib.rnn.BasicRNNCell(num_units=n_hidden, activation=tf.nn.relu) for layer in range(n_layers)]
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers)
model = tf.matmul(outputs, W) + b
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)
outputs = tf.transpose(outputs, [1,0,2])
outputs = outputs[-1]

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
test_batch_size = len(mnist.test.images)
test_xs = mnist.test.images.reshape(test_batch_size, n_step, n_input)
test_ys = mnist.test.labels
print('정확도: ', sess.run(accuracy, feed_dict={X: test_xs, Y: test_ys}))

# LSTM
- https://dgkim5360.tistory.com/entry/understanding-long-short-term-memory-lstm-kr

- 한셀에 가중치가 4개필요
- states 2개

In [ ]:
import numpy as np
values231 = np.array([
    [[1],[2],[3]],
    [[2],[3],[4]]
])

print('입력데이터의 shape',values231.shape) #2x3x1
import tensorflow as tf
tf.reset_default_graph()

#데이터가 하나가 들어오고 100개가 나간다 
tf_values231 = tf.constant(values231,dtype = tf.float32)
#1x100이 4개가 있어야함 
#rnn 모델은 동일 : 망과망을 연결할 때 ( 번역망 : 한글 , 영어 )
#latent time :지연시간 
lstm_cell = tf.contrib.rnn.LSTMCell(num_units =100)
outputs,state = tf.nn.dynamic_rnn(cell=lstm_cell,dtype=tf.float32,
                                 inputs=tf_values231)

print(state.c.shape) # 2x100
print(state.h.shape) # 2x100 

print('rnn이 출력하는 output의 의미:',outputs.shape) #2x3x100

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    output_run,state_run = sess.run([outputs,state])


In [ ]:
tf.reset_default_graph()
values = tf.constant(np.array([
    [[1],[2],[3]],
    [[2],[3],[4]]
]),dtype =tf.float32)

lstm_cell_fw = tf.contrib.rnn.LSTMCell(100)
lstm_cell_bw = tf.contrib.rnn.LSTMCell(100)   # multicell :셀에서  # 양방향 LSTM
(output_fw,output_bw),(output_state_fw,output_state_bw) = tf.nn.bidirectional_dynamic_rnn(
    cell_fw=lstm_cell_fw,cell_bw=lstm_cell_bw,inputs=values,dtype=tf.float32)

print(output_fw.shape)

print(output_bw.shape)

## embedding

In [19]:
# embedding 하는 것 
from tensorflow.contrib import rnn
import pprint
from tensorflow.python.framework import ops
ops.reset_default_graph()
pp = pprint.PrettyPrinter(indent=4) ## pprint : 데이터형태대로 찍기위해서 준비된 함수이다 
sess=tf.InteractiveSession()  #세션을 열고
h=[1,0,0,0]  # 단어들을 one-hot encoding 을 했다 ( 단어를 원핫인코딩=수치화했다 !) 
e=[0,1,0,0]
l=[0,0,1,0]
o=[0,0,0,1] 
with tf.variable_scope('five_sequences') as scope: # variable_scope>>>변수공유를 위해 지정 
    hidden_size = 2  # 4개로 들어와서 2개의 특징으로 추출된다는뜼!!! (4개에서 2개의 특징으로 vector 화 된다)
    
    cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
    x_data = np.array([[h,e,l,l,o]], dtype=np.float32)  
    #[h,e,l,l,o]
    #>>[1,0,0,0]
    #  [0,1,0,0]
    #  [0,0,1,0]
    # [0,0,1,0]
    # [0,0,0,1]  가 됨  >> shape 는 5x4 가 됨   (밖에서 1개를 더 씌어놨기때문에 1x5x4 이다 ) >> 3차원으로 만들기위해서 한개를씌운것이다 
    print(x_data.shape)  # 1x5x4
    
    pp.pprint(x_data)
    
    outputs, states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32) #dynamic_rnn 에 집어넣었다 
    # >> cell의 갯수 : 5
    # 셀당 입력데이터는 4/  셀당 가중치는 4x2 
    
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())  
    print(outputs.shape)##1x5x2
    print(states.shape)## 1x2  ( 데이터 1개가 들어갔으니 1개가 나온다? )
    pp.pprint(states.eval())

(1, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)
array([[[-0.27320525, -0.09965538],
        [-0.43588114, -0.37847227],
        [ 0.74138236,  0.38507318],
        [ 0.08048736,  0.146414  ],
        [-0.49328816,  0.37820202]]], dtype=float32)
(1, 5, 2)
(1, 2)
array([[-0.49328816,  0.37820202]], dtype=float32)


## 알파벳

In [20]:
#텍스트를 코드화하기위해 문자열로
#마지막자 예측

char_arr =['a','b','c','d','e','f','g','h','i','j',
           'k','l','m','n','o','p','q','r','s','t','u','v','w',
          'x','y','z']

num_dic = {n: i for i,n in enumerate(char_arr)} #a #행렬 만드는거
# a:0 dict 형태로 번호를 키로 
dic_len = len(num_dic) # 길이 : 26

#rule base => 자동 툴 => 가중치만 ㅇㅣ용
seq_data=['word','wood','deep','dive','cold','cool','load'
         ,'love','kiss','kind']

#10x3 3은 세글자라서 3
# 10x1 :타겟 


In [21]:
# 100000000000000000000000000000000 : 이런식으로 알파벳 행렬
# 010000000000000000000000000000000 : 26x26
# 001000000000000000000000000000000
# 000100000000000000000000000000000
# 000010000000000000000000000000000


#원핫인코딩하면 전체 데이터가 10개고 한데이터는 3개로 구성되어있다  그리고 그놈이 26개로 되어있다 
#10x3x26

def make_batch(seq_data):
    input_batch = []
    target_batch = []
    for seq in seq_data: # 'word'    #알파벳 인덱스 # w =22,o=24,r=17
        input = [num_dic[n] for n in seq[:-1]] # seq_data에서 마지막에서 한자씩 빼라
        target = num_dic[seq[-1]] # 마지막 글자는 d=3여기서 
        input_batch.append(np.eye(dic_len)[input]) # 원핫 인코딩#dictionary 만들었음
        target_batch.append(target)
    return input_batch,target_batch

In [22]:
learning_rate =0.01
n_hidden = 128
total_epoch = 30   # 30번 돌아간다
n_step = 3 
n_input = n_class =dic_len  # 전부 26개다 

X = tf.placeholder(tf.float32, [None, n_step, n_input])  # 들어오는 데이터가 10x3x26 으로 들어온다 
Y = tf.placeholder(tf.int32, [None])  # Y 는 10x26 으로 들어온다  
#FFNN
W = tf.Variable(tf.random_normal([n_hidden,n_class])) # 128 x 26
b = tf.Variable(tf.random_normal([n_class])) # 26

In [23]:
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden) # n_hidden 을 basicLSTMCEll 에 줬으니까 특성수는 128이 된다 
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob = 0.5, seed=10) # dropoutwrapper >> 윗줄에서 계산한양이 많으니까 줄여준다 # 50퍼센트만 남긴다 ( 0.5) >> 과적합을 방지하기 위해서 dropout 을 한다 

cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])  # 2개의 layer 로 구성된 멀티셀을 만든다 
outputs, states = tf.nn.dynamic_rnn(multi_cell,X, dtype=tf.float32)  # 다이내믹 rnn 에 집어넣는다 >> state 와 output 이 나온다 
                                                                        # output 으로 나오는 놈 >> 10x3x128
                                                                      # state 로 나오는 놈들 >> 10x128 개로 나온다 

In [24]:
outputs = tf.transpose(outputs, [1,0,2]) # 3x10x 128 이 된다 >> 맨 뒷장만 뺌
outputs = outputs[-1]    # 맨뒤의 셀에서 나온 10x128 만 남는다 
model = tf.matmul(outputs, W) +b      # 비선형 회귀방식으로 문제를 풀고있다  

# 분류에서 entropy 식이다 
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y)) #
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [27]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
input_batch,target_batch = make_batch(seq_data)  #SEQ_DATA : 단어데이터

for epoch in range(total_epoch):
    _,loss = sess.run([optimizer,cost],
                     feed_dict={X:input_batch,Y:target_batch})
    print('Epoch:','%04d'%(epoch+1),
         'cost=','{:.6f}'.format(loss))

Epoch: 0001 cost= 2.824239
Epoch: 0002 cost= 2.100840
Epoch: 0003 cost= 1.485738
Epoch: 0004 cost= 0.780827
Epoch: 0005 cost= 0.931172
Epoch: 0006 cost= 0.617634
Epoch: 0007 cost= 0.396059
Epoch: 0008 cost= 0.359609
Epoch: 0009 cost= 0.308816
Epoch: 0010 cost= 0.523936
Epoch: 0011 cost= 0.269840
Epoch: 0012 cost= 0.084347
Epoch: 0013 cost= 0.290791
Epoch: 0014 cost= 0.381600
Epoch: 0015 cost= 0.100686
Epoch: 0016 cost= 0.065231
Epoch: 0017 cost= 0.114521
Epoch: 0018 cost= 0.081495
Epoch: 0019 cost= 0.129232
Epoch: 0020 cost= 0.093834
Epoch: 0021 cost= 0.046559
Epoch: 0022 cost= 0.039512
Epoch: 0023 cost= 0.023533
Epoch: 0024 cost= 0.014485
Epoch: 0025 cost= 0.022424
Epoch: 0026 cost= 0.006681
Epoch: 0027 cost= 0.067299
Epoch: 0028 cost= 0.020038
Epoch: 0029 cost= 0.003289
Epoch: 0030 cost= 0.004081


In [29]:
prediction = tf.cast(tf.argmax(model,1),tf.int32)
prediction_check= tf.equal(prediction,Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check,tf.float32))

input_batch,target_batch = make_batch(seq_data)
predict,accuracy_val = sess.run([prediction,accuracy],
                               feed_dict={X:input_batch , Y:target_batch})

## 문제
- predict한 결과를 출력하고 
- accuracy를 출력하시오
- 입력단어와 예측 결과를 나란히 출력해 보시오 

In [36]:
#1 
print('예측 결과 :',predict)

#2
print('정확도', accuracy_val)

#3
predict_words =[]
#단어가 순서대로 입력, 순서대로 예측
for idx,  val in enumerate(seq_data):
    last_char = char_arr[predict[idx]] #인덱스가 단어 매핑
    predict_words.append(val[:3]+last_char)

print('입력값:',[w[:3] + ' 'for w in seq_data])
print("예측값:",predict_words)
print('정확도:',accuracy_val)

예측 결과 : [ 3  3 15  4  3 11  3  4 18  3]
정확도 1.0
입력값: ['wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
예측값: ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
정확도: 1.0
